In [1]:
from glob import glob
import numpy as np
data_path = "/Users/soltan/Programs/kaggle/raw_data/train/train/*"

import librosa
def extract_features(file_name):
    n_fft = 2048
    hop_length = 512
    signal, sr = librosa.load(file_name, sr=None)
    mfcc = librosa.feature.mfcc(signal, n_fft=n_fft, hop_length=hop_length, n_mfcc=13).flatten()
    return mfcc

In [2]:
from tqdm import tqdm
data_dir = np.array(glob(data_path))
features, labels = [], []
for file in tqdm(data_dir):
    file_name = file.split("/")[-1]
    file_name = file.split(".")[0]
    name, label = file_name.split("-")[0], file_name.split("-")[1]
    features.append(extract_features(file))
    labels.append(label)


100%|██████████| 9000/9000 [00:59<00:00, 151.51it/s]


In [3]:
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, targets_train, targets_test = train_test_split(features, labels, test_size=0.2)


In [17]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=17)
model.fit(inputs_train, targets_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=17, p=2,
                     weights='uniform')

In [21]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

model = GridSearchCV(KNeighborsClassifier(), grid_params, n_jobs=10, verbose=1000)
model.fit(inputs_train, targets_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:   12.1s
[Parallel(n_jobs=10)]: Done   2 tasks      | elapsed:   13.2s
[Parallel(n_jobs=10)]: Done   3 tasks      | elapsed:   14.1s
[Parallel(n_jobs=10)]: Done   4 tasks      | elapsed:   15.6s
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   16.8s
[Parallel(n_jobs=10)]: Done   6 tasks      | elapsed:   18.0s
[Parallel(n_jobs=10)]: Done   7 tasks      | elapsed:   19.4s
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:   20.5s
[Parallel(n_jobs=10)]: Done   9 tasks      | elapsed:   21.9s
[Parallel(n_jobs=10)]: Done  10 tasks      | elapsed:   23.4s
[Parallel(n_jobs=10)]: Done  11 tasks      | elapsed:   29.4s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   30.7s
[Parallel(n_jobs=10)]: Done  13 tasks      | elapsed:   32.2s
[Parallel(n_jobs=10)]: Done  14 tasks      | elapsed:  

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=10,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9, 11, 15],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1000)

In [22]:
from sklearn import metrics
predictions = model.predict(inputs_test)
print("Accuracy:",metrics.accuracy_score(targets_test, predictions))


Accuracy: 0.5611111111111111
